In [1]:
# import sys

# !git clone https://github.com/TimeBinFM/binconvfm.git
# %cd binconvfm
# !{sys.executable} -m pip install .

## Base dataset preparation

In [2]:
import datasets
import torch
from preprocessing.common import TensorIterableDataset

class GiftEvalWindowedDataset(TensorIterableDataset):
    def __init__(self, dataset_name: str, file_names: list[str], window_size: int, 
        prediction_depth: int, step: int, batch_size: int, pre_batch_timeseries_count: int):
        super().__init__()

        self.dataset_name = dataset_name
        self.file_names = file_names
        self.window_size = window_size
        self.prediction_depth = prediction_depth
        self.batch_size = batch_size
        self.step = step
        self.pre_batch_timeseries_count = pre_batch_timeseries_count
        self.dataset = self._build_dataset()

    def _build_dataset(self):
        return (
            datasets.load_dataset(
                self.dataset_name,
                split='train',
                data_files=self.file_names,
                streaming=True,
            )
            .select_columns(["target"])
            .batch(self.pre_batch_timeseries_count)
            .map(self._to_window_tensors)
        )
        
    def _item_to_window_tensor(self, target):
        target_tensor = torch.tensor(target)
        target_tensor_shape = target_tensor.shape

        unfold_dimension = len(target_tensor_shape) - 1 
        
        window_tensor = target_tensor.unfold(dimension=unfold_dimension, size=self.window_size+self.prediction_depth, step=self.step)
        
        window_tensor_shape = window_tensor.shape
        result_tensor = window_tensor if len(window_tensor_shape) == 2 else window_tensor.flatten(0, 1)

        return result_tensor

    def _to_window_tensors(self, batch):
        tensors = [self._item_to_window_tensor(item) for item in batch['target']]
        total_tensor = torch.cat(tensors)
        return {
            'target': torch.split(total_tensor, self.batch_size)
        }
        
    def __iter__(self):
        for dict in self.dataset:
            for batch in dict['target']:
                yield batch


In [3]:
from binconvfm.utils.download.gift_eval_windowed_dataset import GiftEvalWindowedDataset
from binconvfm.utils.download.gift_eval_util import get_file_names_per_dataset

dataset_name = "Salesforce/GiftEvalPretrain"
ds_name = 'buildings_900k'

file_names_per_dataset = get_file_names_per_dataset(dataset_name)

files_per_ds = 8

# batch_size=8192, 

def get_ds(file_names, window_size=32, prediction_depth=1, step=1, batch_size=8192, pre_batch_ts_count=100):
    return GiftEvalWindowedDataset(
        dataset_name,
        file_names,
        window_size,
        prediction_depth,
        step,
        batch_size,
        pre_batch_ts_count
    )

# ds1 = get_ds('buildings_900k')
file_names = file_names_per_dataset['borg_cluster_data_2011']#[:files_per_ds]
total_ds = get_ds(file_names)

['borg_cluster_data_2011/data-00000-of-00031.arrow']
['borg_cluster_data_2011/data-00001-of-00031.arrow']
['borg_cluster_data_2011/data-00002-of-00031.arrow']
['borg_cluster_data_2011/data-00003-of-00031.arrow']
['borg_cluster_data_2011/data-00004-of-00031.arrow']
['borg_cluster_data_2011/data-00005-of-00031.arrow']
['borg_cluster_data_2011/data-00006-of-00031.arrow']
['borg_cluster_data_2011/data-00007-of-00031.arrow']
['borg_cluster_data_2011/data-00008-of-00031.arrow']
['borg_cluster_data_2011/data-00009-of-00031.arrow']
['borg_cluster_data_2011/data-00010-of-00031.arrow']
['borg_cluster_data_2011/data-00011-of-00031.arrow']
['borg_cluster_data_2011/data-00012-of-00031.arrow']
['borg_cluster_data_2011/data-00013-of-00031.arrow']
['borg_cluster_data_2011/data-00014-of-00031.arrow']
['borg_cluster_data_2011/data-00015-of-00031.arrow']
['borg_cluster_data_2011/data-00016-of-00031.arrow']
['borg_cluster_data_2011/data-00017-of-00031.arrow']
['borg_cluster_data_2011/data-00018-of-00031.a

Resolving data files:   0%|          | 0/31 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/31 [00:00<?, ?it/s]

In [5]:
from binconvfm.utils.download.window_mixing_dataset import WindowMixingDataset

mixing_dataset = WindowMixingDataset(total_ds, prediction_depth=1, seed=42, batch_size=8192, prefetch_depth=1024)

## Model preparation

In [6]:
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F
import pytorch_lightning as pl
from torch import nn

class LinearRegressionModel(pl.LightningModule):
    def __init__(self):
        super().__init__()
        self.linear = nn.Linear(32, 1)

    def forward(self, x):
        return self.linear(x)  # Output shape: [batch_size, 1]

    def training_step(self, batch, batch_idx):
        x, y = batch
        x = x.float()
        y = y.float()
        y_hat = self(x)
        loss = F.mse_loss(y_hat, y)
        self.log("train_loss", loss)
        return loss

    def configure_optimizers(self):
        # GPU-accelerated AdamW (fused)
        return torch.optim.AdamW(self.parameters(), lr=1e-3, fused=True)

## Train a model

In [ ]:
import torch
import pytorch_lightning as pl
from pytorch_lightning.profilers import PyTorchProfiler
from torch.utils.data import DataLoader

# ----------------------------
# Dataset and DataLoader
# ----------------------------
# window_tensor_dataset must be a torch.Tensor dataset or TensorDataset
dataloader = DataLoader(
    mixing_dataset,
    shuffle=False,
    batch_size=None,
    num_workers=8,        # CPU parallelism for data loading
    pin_memory=True,       # speeds up CPU->GPU transfer
    persistent_workers=True,  # keep workers alive between epochs
    prefetch_factor=1000,     # prefetch batches per worker
)

# ----------------------------
# Trainer
# ----------------------------
trainer = pl.Trainer(
    max_epochs=1,
    accelerator="gpu",         # force GPU
    devices=1,                 # single GPU
    precision=16,              # automatic mixed precision for speed
    gradient_clip_val=0.0,     # avoid gradient clipping overhead
)

# ----------------------------
# Model to GPU
# ----------------------------
model = LinearRegressionModel().to("cuda")

# ----------------------------
# Training
# ----------------------------
trainer.fit(model, dataloader)

Using 16bit Automatic Mixed Precision (AMP)
💡 Tip: For seamless cloud uploads and versioning, try installing [litmodels](https://pypi.org/project/litmodels/) to enable LitModelCheckpoint, which syncs automatically with the Lightning model registry.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name   | Type   | Params | Mode 
------------------------------------------
0 | linear | Linear | 33     | train
------------------------------------------
33        Trainable params
0         Non-trainable params
33        Total params
0.000     Total estimated model params size (MB)
1         Modules in train mode
0         Modules in eval mode


Training: |          | 0/? [00:00<?, ?it/s]